In [ ]:
!wget https://github.com/KhanhPham2411/ml-finance/archive/master.zip

In [ ]:
!unzip master.zip

In [5]:
cd ml-finance-master/examples/0-deep-q-learning-3-stocks/

/content/ml-finance-master/examples/0-deep-q-learning-3-stocks


In [1]:
import pickle
import time
import numpy as np
import re
import matplotlib.pyplot as plt
import pandas as pd
import glob
import os
from statistics import median

from envs import TradingEnv
from agent import DQNAgent
from utils import get_data, get_scaler

# Auto reload
%load_ext autoreload
%autoreload 2

ModuleNotFoundError: ignored

In [2]:
data = np.around(get_data())
train_data = data[:, :3526]
test_data = data[:, 3526:]

NameError: ignored

In [ ]:
fig = plt.figure()
plt.plot(train_data.flatten())
fig.suptitle('train_data', fontsize=20)
fig = plt.figure()
plt.plot(test_data.flatten())
fig.suptitle('test_data', fontsize=20)
plt.show()

In [ ]:
episode=2000
batch_size=32
initial_invest=20000

In [ ]:
env = TradingEnv(train_data, initial_invest)
state_size = env.observation_space.shape
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)
scaler = get_scaler(env)
train_results = pd.DataFrame()

## Train

In [ ]:
for e in range(episode):
    state = env.reset()
    state = scaler.transform([state])
    for i in range(env.n_step):
        action = agent.act(state)
        next_state, reward, done = env.step(action)
        next_state = scaler.transform([next_state])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print(".", end='')
            if (e + 1) % 20 == 0:
                print("")
                print("episode: {}/{}, median profit so far: {}".format(
                    e + 1, episode, median(train_results['value'].tolist())))
            train_results = train_results.append(env._stats(), ignore_index=True)
            break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
    if (e + 1) % 10 == 0:
        agent.save('weights/{}-dqn.h5'.format(time.strftime('%Y%m%d%H%M')))

In [ ]:
train_results['profit'] = train_results['value'] - initial_invest
columns_to_plot = ['win_loss_ratio', 'profit', 'trade_count']
fig, axes = plt.subplots(nrows=1,ncols=len(columns_to_plot), figsize=(16,6))
for i in range(len(columns_to_plot)):
    axes[i].title.set_text(columns_to_plot[i])
    train_results[columns_to_plot[i]].plot(ax=axes[i], subplots=True)
plt.show()

## Test

In [ ]:
state_size = env.observation_space.shape
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)
scaler = get_scaler(env)
test_portfolio_value = []

In [ ]:
# remake the env with test data
env = TradingEnv(test_data, initial_invest)
# load trained weights from latest saved
latest_weights_file = max(glob.glob('weights/*.h5'), key=os.path.getctime)
print('loading weights file: {0}'.format(latest_weights_file))
agent.load(latest_weights_file)
test_results = pd.DataFrame()

In [ ]:
for e in range(episode):
    state = env.reset()
    state = scaler.transform([state])
    for i in range(env.n_step):
        action = agent.act(state)
        next_state, reward, done = env.step(action)
        next_state = scaler.transform([next_state])
        state = next_state
        if done:
            test_results = test_results.append(env._stats(), ignore_index=True)
            break

In [ ]:
test_results['profit'] = test_results['value'] - initial_invest

In [ ]:
print('Median profit: {0} ({1}%)'.format(
    test_results['profit'].median(),
    test_results['profit'].median() / initial_invest * 100
))

In [ ]:
test_results.hist(column='profit', bins=60)